# Data Transformation to feed finetuning.py

In [1]:
import pandas as pd

# data_path_cancer = "../training_set_anxiety_encore_plus_full.csv"
# data_path_cancer = "../training_set_antivirals.csv"
data_path_cancer = "../ssris_dataset_std_smiles.csv"
data_cancer = pd.read_csv(data_path_cancer, sep=",", index_col=0)
data_cancer.head()

# data_cancer['against_cancer'] = data_cancer['against_anxiety'].fillna(0)
data_cancer["classe"] = data_cancer["against_depression"].apply(lambda x: 1 if x else 0)
# data_cancer["classe"] = data_cancer["against_virus"].apply(lambda x: 1 if x else 0)

# data_cancer["classe"] = data_cancer["against_depression"].apply(lambda x: 1 if x else 0)
data_cancer = data_cancer.rename(columns={"classe": "label"})
# data_cancer = data_cancer.iloc[:, 2:]
data_cancer = data_cancer.iloc[:, [0, 2]]
data_cancer.head()

,smiles,label
0,CNC1CCC(c2ccccc21)c1cc(Cl)c(Cl)cc1,1
1,CNCCC(Oc1ccc(cc1)C(F)(F)F)c1ccccc1,1
2,CN(C)CCCC1(OCc2cc(ccc12)C#N)c1ccc(F)cc1,1
3,CN(C)CCCN1c2ccccc2CCc2ccc(Cl)cc12,1
4,Cc1cc(C)c(cc1)Sc1ccccc1N1CCNCC1,1


In [3]:
save_path = './SMILES-BERT/Datasets/training.csv'
data_cancer.to_csv(save_path, index=False)

from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(data_cancer, test_size=0.10, random_state=42)
train_df, eval_df = train_test_split(train_df, test_size=0.2, random_state=42)
save_train_path = './SMILES-BERT/Datasets/train.csv'
train_df.to_csv(save_train_path, index=False)
save_test_path = './SMILES-BERT/Datasets/test.csv'
test_df.to_csv(save_test_path, index=False)
save_eval_path = './SMILES-BERT/Datasets/eval.csv'
eval_df.to_csv(save_eval_path, index=False)

num_label_1 = train_df[train_df["label"] == 1].shape[0]
num_label_2 = eval_df[eval_df["label"] == 1].shape[0]
num_label_3 = test_df[test_df["label"] == 1].shape[0]

print(f"\nNombre de labels 1 dans l'ensemble d'entraînement: {num_label_1}")
print(f"Nombre de labels 1 dans l'ensemble d'éval: {num_label_2}")
print(f"Nombre de labels 1 dans l'ensemble de test: {num_label_3}")




Nombre de labels 1 dans l'ensemble d'entraînement: 37
Nombre de labels 1 dans l'ensemble d'éval: 11
Nombre de labels 1 dans l'ensemble de test: 4


# Data Transformation of COCONUT to feed predict.py

In [4]:
df_training = "../ssris_dataset_std_smiles.csv"
df_training = pd.read_csv(df_training, sep=",", index_col=0)
df_training['smiles_length'] = df_training['smiles'].apply(len)

mean_length = df_training['smiles_length'].mean()
std_length = df_training['smiles_length'].std()

lower_bound = mean_length - 1 * std_length
upper_bound = mean_length + 1 * std_length
df_training.head()

,smiles,against_depression,smiles_length
0,CNC1CCC(c2ccccc21)c1cc(Cl)c(Cl)cc1,True,34
1,CNCCC(Oc1ccc(cc1)C(F)(F)F)c1ccccc1,True,34
2,CN(C)CCCC1(OCc2cc(ccc12)C#N)c1ccc(F)cc1,True,39
3,CN(C)CCCN1c2ccccc2CCc2ccc(Cl)cc12,True,33
4,Cc1cc(C)c(cc1)Sc1ccccc1N1CCNCC1,True,31


In [5]:
import pandas as pd

df_coconut = "../COCONUT_DB_std_smiles.csv"
df = pd.read_csv(df_coconut, sep=",", index_col=0)
print(len(df))

# Calculer la longueur de chaque chaîne dans la colonne "smiles"
df['smiles_length'] = df['smiles'].apply(len)

filtered_df = df[(df['smiles_length'] >= lower_bound) & (df['smiles_length'] <= upper_bound)]

# Si vous souhaitez supprimer la colonne 'smiles_length' après le filtrage, vous pouvez le faire
filtered_df = filtered_df.drop(columns=['smiles_length'])
print(len(filtered_df))

save_path = '../COCONUT_DB_std_smiles_filtered.csv'
filtered_df.to_csv(save_path, index=False)

407270
149962
